In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

data_train = pd.read_csv('/content/drive/MyDrive/ML/imageset/ECG_train.csv')
data_test = pd.read_csv('/content/drive/MyDrive/ML/imageset/ECG_test.csv')

Read dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

data_train = pd.read_csv('/content/drive/MyDrive/ML/imageset/ECG_train.csv')
data_test = pd.read_csv('/content/drive/MyDrive/ML/imageset/ECG_test.csv')

Data preprocessing

In [ ]:
data_train.dropna(axis=1,inplace=True)
data_test.dropna(axis=1,inplace=True)
y_train = data_train.values[:,0]
X_train = data_train.values[:,1:5423]
y_test = data_test.values[:,0]
X_test = data_test.values[:,1:5423]
X_test

Shuffle training set and testing set

In [ ]:
from sklearn.model_selection import train_test_split
#X_train,X_test,y_train,y_test=train_test_split(X_std,y,test_size=0.3)
np.random.seed(200)
np.random.shuffle(X_train) 
np.random.seed(200)
np.random.shuffle(y_train)
 
np.random.seed(200)
np.random.shuffle(X_test) 
np.random.seed(200)
np.random.shuffle(y_test)
y_test

In [ ]:
# 构建CNN模型
def buildModel():
    newModel = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(5422, 1)),
        # 第一个卷积层, 4 个 21x1 卷积核
        tf.keras.layers.Conv1D(filters=32, kernel_size=16, strides=1, padding='SAME', activation='relu'),
        # 第一个池化层, 最大池化,4 个 3x1 卷积核, 步长为 2
        tf.keras.layers.MaxPool1D(pool_size=200, strides=3, padding='SAME'),

        #tf.keras.layers.Dropout(0.2),

        # 第二个卷积层, 16 个 23x1 卷积核
        tf.keras.layers.Conv1D(filters=64, kernel_size=32, strides=1, padding='SAME', activation='relu'),
        # 第二个池化层, 最大池化,4 个 3x1 卷积核, 步长为 2
        tf.keras.layers.MaxPool1D(pool_size=200, strides=3, padding='SAME'),

        #tf.keras.layers.Dropout(0.2),

        # 打平层,方便全连接层处理
        tf.keras.layers.Flatten(),
        # 全连接层,128 个节点
        tf.keras.layers.Dense(128, activation='relu'),
        # Dropout层,dropout = 0.2
        tf.keras.layers.Dropout(rate=0.2),
        # 全连接层,2 个节点
        tf.keras.layers.Dense(1, activation='sigmoid')
        
    ])

    return newModel

Show details of CNN model

In [ ]:
model = buildModel()
model.summary()

Set learning rate

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K
def scheduler(epoch):
  
    lr = 0.0001
    K.set_value(model.optimizer.lr, lr )
    print("lr changed to {}".format(lr ))
    return K.get_value(model.optimizer.lr)

reduce_lr = LearningRateScheduler(scheduler)

Train model and plot curve of loss and accuracy

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import keras


X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape, y.shape)
#reduce_lr = LearningRateScheduler(scheduler)
model = buildModel()
model.summary()

model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
#kfold = KFold(n_splits=10,shuffle=True,random_state=0)
hist = model.fit(X_train, y_train, epochs=100,
                  batch_size=72,
                  #validation_split=0.3,
                  shuffle=True,
                  callbacks=[reduce_lr]
                  )

#hist = model.fit(X_train, y1, batch_size=64, validation_split = 0.3, epochs=30, shuffle=True, verbose=1)

plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['accuracy'], label='acc')
#plt.plot(hist.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()

Evaluate model in testing set

In [ ]:
model.evaluate(X_test, y_test, batch_size=32, verbose=1)

Predict testing set (1 : positive , 0 : negative)

In [ ]:
Y_pred = model.predict(X_test)
Y_pred = np.int64(Y_pred>0.5)
Y_pred